## Advanced Machine Learning: FFNN

#### Import Statements

In [2]:
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.metrics import accuracy_score, mean_squared_error
from IPython.display import clear_output

#### Load Dataset

In [3]:
def load(filename):		
    data = arff.loadarff(filename)
    df = pd.DataFrame(data[0])
    # Convert string attribute to integer
    df.outlook = pd.Categorical(pd.factorize(df.outlook)[0])
    df.outlook = pd.to_numeric(df.outlook, errors='coerce')
    df.windy = pd.Categorical(pd.factorize(df.windy)[0])
    df.windy = pd.to_numeric(df.windy, errors='coerce')
    df.play = pd.Categorical(pd.factorize(df.play)[0])
    df.play = pd.to_numeric(df.play, errors='coerce')

    df.head()
    return df

#### Split Train and Test Data

In [288]:
def split(data, test_size=0.1):
    y = data.play
    x = data.drop('play',axis=1)
#     scaler = preprocessing.MinMaxScaler().fit(x)
    x = scaler.transform(x)
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    return X_train, X_test, y_train, y_test


### FFNN Module

In [327]:
class FFNN: 
    def __init__(self, nb_feature, nb_hidden_layer, nb_nodes):
        self.nb_output_layer = 1
        self.nb_feature = nb_feature
        self.nb_hidden_layer = nb_hidden_layer
        self.nb_nodes = nb_nodes
        self.nb_weight_layer = nb_hidden_layer + 1

        self.weights = []
        self.bias = []
        self.output_neurons = []
        self.feature_neurons = []
        self.hidden_neurons = []

    def sigmoid(self, x):
        return 1/(1 + np.exp(-x)) 

    def count_error(self, learning_rate, target_output):
        return pow((learning_rate - target_output),2)/2

    def init_weights(self):
        for i in range (self.nb_weight_layer):
            temp_weight = []
            if i == 0:
                temp_weight = np.random.randn(self.nb_feature, self.nb_nodes)
            elif i < self.nb_weight_layer - 1:
                temp_weight = np.random.randn(self.nb_nodes, self.nb_nodes)
            elif i == self.nb_weight_layer - 1:
                temp_weight = np.random.randn(self.nb_nodes, self.nb_output_layer)
            self.weights.append(temp_weight)
            self.bias.append(np.random.random())

    def get_sigmoid_value(self, m=None, n=None, output=False):
        if output:
            m = self.nb_weight_layer-1
            n = 0
        if m == 0:
            previous_layer = self.feature_neurons  
        else :
            previous_layer = self.hidden_neurons[m-1]
        weight_layer = self.weights[m]
        sigmoid_value = 0

        for i in range(len(previous_layer)):
            sigmoid_value += previous_layer[i] * weight_layer[i, n]
        sigmoid_value = self.sigmoid(sigmoid_value + self.bias[m])
        return sigmoid_value

    def get_output(self, input):
        self.feature_neurons = []
        self.hidden_neurons = []
        for item in input:
            self.feature_neurons.append(item)

        for i in range(self.nb_hidden_layer):
            temp_hidden_layer = []
            for j in range(self.nb_nodes):
                temp_hidden_layer.append(self.get_sigmoid_value(i, j))
            self.hidden_neurons.append(temp_hidden_layer)
        output = self.get_sigmoid_value(output=True)
        return output

    def fit(self, x_train, y_train, batch_size, momentum=0.001, learning_rate=0.5, epoch=5):
        self.init_weights()
        for i in range(epoch):
            #masuk ke epoch
            for	j in range(0, len(y_train), batch_size):
                #masuk ke batch
                x_mini = x_train[j:j+batch_size]
                y_mini = y_train[j:j+batch_size]
                outputs = 0
                for x,y in zip(x_mini,y_mini):
                    self.output_neurons.append(self.get_output(x)-y)
                avg_error = np.average(self.output_neurons)
                    # Update weight with gradient descent
                self.update_weight(learning_rate, momentum, j, i, avg_error)
                self.output_neurons = []             
    
    def update_weight(self, learning_rate, momentum, batch, epoch, outputs):
        previous_layer = []
        next_layer = []
        
        for i in reversed(range(self.nb_weight_layer)):
            previous_weight = self.weights[i]
            if (i == 0):
                previous_layer = self.feature_neurons
            else:
                previous_layer = self.hidden_neurons[i-1]
                
            if (i == self.nb_weight_layer-1):
                next_layer = outputs
            else:
                next_layer = self.hidden_neurons[i]

                
            prevm = np.matrix(previous_layer)
            nextm = np.matrix(next_layer)
            
            self.bias[i] -= learning_rate*(outputs) + momentum*self.bias[i]
            self.weights[i] -= (1/(i+1))*learning_rate*(prevm.T.dot(nextm)) + momentum*self.weights[i]
        
    
    def predict(self, X):
        Y_pred = []
        
        for x in X:
#             print(self.get_output(x))
            Y_pred.append(self.get_output(x))

        return Y_pred


if __name__ == "__main__":
    data = load('weather.arff')
    X_train, X_test, y_train, y_test = split(data, 0.05)
    ffnn = FFNN(X_train.shape[1],2,3)
    ffnn.fit(X_train, y_train, batch_size=2, epoch=100)
    Y_pred = ffnn.predict(X_test)
    Ytrain_pred = ffnn.predict(X_train)
#     print("Test", X_test)
#     print("Train", X_train)

    y_pred_binarised_train = []
    for item in Ytrain_pred:
        if (item >= 0.5):
            y_pred_binarised_train.append(1)
        else:
            y_pred_binarised_train.append(0)
            
    print(y_pred_binarised_train)
    print("Train Accuracy: ",accuracy_score(y_pred_binarised_train, y_train))

    y_pred_binarised_test = []
    for item in Y_pred:
        if (item >= 0.5):
            y_pred_binarised_test.append(1)
        else:
            y_pred_binarised_test.append(0)
            
    print(y_pred_binarised_test)
    print("Test Predict Accuracy: ",accuracy_score(y_pred_binarised_test, y_test))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Train Accuracy:  0.75
[1, 1]
Test Predict Accuracy:  0.0
